<a href="https://colab.research.google.com/github/ChaitaliV/Objective-criterias-to-quantify-the-accuracy-of-explanation/blob/main/Metrices/Proximity_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install lime
!pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283859 sha256=8cd20cfb6780c67e1a9c886c69aa21ece89b49dfe97a0f43d3e6268207ff49ae
  Stored in directory: /root/.cache/pip/wheels/ed/d7/c9/5a0130d06d6310bc6cbe55220e6e72dcb8c4eff9a478717066
Successfully built lime
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 kB 11.

In [3]:
import torch
from lime.lime_text import LimeTextExplainer
from transformers import pipeline, BertTokenizer
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
import pandas as pd
import shap
import scipy as sp
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-7025d32d-4f5b-28a8-aa3d-26997127162e)


In [5]:
model=torch.load(r'/content/drive/MyDrive/Saved Models/MuRIL.pt',map_location=torch.device('cuda'))
tokenizer = BertTokenizer.from_pretrained('google/muril-base-cased', do_lower_case=True)

In [33]:
df = pd.read_csv(r'/content/drive/MyDrive/test.csv')

In [8]:
batch1 = df['hindi_text'][:50]
batch2 = df['hindi_text'][50:]

In [9]:
# Define a function to calculate the distance between two sets of features
def jaccard_distance(a, b):
    a = set(a)
    b = set(b)
    return 1 - len(a.intersection(b)) / len(a.union(b))

## LIME

In [16]:
def adapter(data):
  #inputs = tokenizer(data, return_tensors="pt")
  inputs = tokenizer(data, 
          return_tensors='pt', 
          padding=True, 
          truncation=True, 
          max_length=128)

  with torch.no_grad():
      #m = model(**inputs)
      #print(model(**inputs).logits)
      logits = model(**inputs).logits
      scores = model(**inputs)[0].softmax(1).detach().cpu().numpy()

  #predicted_class_id = logits.argmax().item()
  # classifications = ['Introduction','Depression','Grey Area']
  # classification = logits.argmax().item()
  # class_name = classifications[classification]
  
  return scores

def pre_process(data):
  inputs = tokenizer(data, 
            return_tensors='pt', 
            padding=True, 
            truncation=True, 
            max_length=128)
  return inputs

def predictor(data):
  #inputs = tokenizer(data, return_tensors="pt")
  inputs = tokenizer(data, 
          return_tensors='pt', 
          padding=True, 
          truncation=True, 
          max_length=128)

  with torch.no_grad():
      #m = model(**inputs)
      #print(model(**inputs).logits)
      logits = model(**inputs).logits
      scores = model(**inputs)[0].softmax(1).detach().cpu().numpy()

  predicted_class_id = logits.argmax().item()
  classifications = ['Introduction','Depression','Grey Area']
  classification = logits.argmax().item()
  class_name = classifications[classification]
  
  return class_name


In [17]:
class_name = ['Introduction','Depression','Grey Area']
explainer = LimeTextExplainer(class_names = class_name)

In [ ]:
def proximity_score_lime(text):
  data = {}
  exp = explainer.explain_instance(text,adapter)
  exp_words = ' '.join([x[0] for x in exp.as_list()])
  data['predicted_label'] = predictor(text)
  data['explanation_label'] = predictor(exp_words)

  original_text = pre_process(text).to('cuda')
  exp_text = pre_process(exp_words).to('cuda')

  ## get the embeddings from the model 
  input_embedding = model.module.get_input_embeddings()(original_text['input_ids'])[0].unsqueeze(0)[0]
  explanation_embedding = model.module.get_input_embeddings()(exp_text['input_ids'])[0].unsqueeze(0)[0]

  # Convert the embeddings to numpy arrays and calculate cosine similarity
  input_embedding = input_embedding.detach().cpu().numpy().reshape(1, -1)
  explanation_embedding = explanation_embedding.detach().cpu().numpy().reshape(1, -1)
  max_len = max(input_embedding.shape[1],explanation_embedding.shape[1])
  input_embedding = np.pad(input_embedding, ((0, 0), (0, max_len - input_embedding.shape[1])), 'constant')
  explanation_embedding = np.pad(explanation_embedding, ((0,0),(0, max_len - explanation_embedding.shape[1])), 'constant')

  proximity_score = cosine_similarity(input_embedding, explanation_embedding)
  data['proximity score'] = proximity_score[0][0]
  return data
  
### proximity score was calculated for first 67 sentences

In [ ]:
Data = []

In [ ]:
for sent in batch2:
  Data.append(proximity_score_lime(sent))

In [ ]:
df3 = pd.DataFrame(Data)
df3.to_csv('proximity_lime_2.csv')

## SHAP

In [10]:

## SHAP adaptor and predictor
def adapter_shap(data):
  #inputs = tokenizer(data, return_tensors="pt")
  inputs = tokenizer(data, 
          return_tensors='pt', 
          padding=True, 
          truncation=True, 
          max_length=128)

  with torch.no_grad():
      logits = model(**inputs).logits
      scores = model(**inputs)[0].softmax(1).detach().cpu().numpy()

  predicted_class_id = logits.argmax().item()
  classifications = ['Introduction','Depression','Grey Area']
  classification = logits.argmax().item()
  class_name = classifications[classification]
  return class_name

def predictor_shap(x):
    probas = model(**tokenizer(x, return_tensors="pt", padding=True))[0].softmax(1).detach().cpu().numpy()
    # logits = model(**tokenizer(x, return_tensors="pt", padding=True)).logits
    # predicted_class_id = logits.argmax().item()
    # classifications = ['Introduction','Depression','Grey Area']
    # classification = logits.argmax().item()
    # class_name = classifications[classification]
    # print(class_name)
    s = sp.special.logit(probas[:,1])
    return s

def f_batch(x):
    val = np.array([])
    for i in x:
      val = np.append(val, predictor_shap(i))
    return val

In [21]:
explainer = shap.Explainer(f_batch, tokenizer)
def explain(data):
  test = { 'text': [data]}
  shap_values = explainer(test)
  return shap_values

In [29]:
def proximity_score_shap(text):
  data = {}
  test = { 'text': [text]}
  shap_values = explainer(test)
  # Get the SHAP values and original text features for the current example
  values = shap_values[0].values
  features = shap_values[0].data

  # Extract the important features from the explanation
  threshold = np.abs(values).mean()
  important_features = features[np.abs(values) >= threshold]
  exp_text = " ".join(important_features)
  data['predicted_label'] = predictor(text)
  data['explanation_label'] = predictor(exp_text)
  original_text = pre_process(text).to('cuda')
  exp_text = pre_process(exp_text).to('cuda')

  ## get the embeddings from the model 
  input_embedding = model.module.get_input_embeddings()(original_text['input_ids'])[0].unsqueeze(0)[0]
  explanation_embedding = model.module.get_input_embeddings()(exp_text['input_ids'])[0].unsqueeze(0)[0]

  # Convert the embeddings to numpy arrays and calculate cosine similarity
  input_embedding = input_embedding.detach().cpu().numpy().reshape(1, -1)
  explanation_embedding = explanation_embedding.detach().cpu().numpy().reshape(1, -1)
  max_len = max(input_embedding.shape[1],explanation_embedding.shape[1])
  input_embedding = np.pad(input_embedding, ((0, 0), (0, max_len - input_embedding.shape[1])), 'constant')
  explanation_embedding = np.pad(explanation_embedding, ((0,0),(0, max_len - explanation_embedding.shape[1])), 'constant')
  proximity_score = cosine_similarity(input_embedding, explanation_embedding)
  data['proximity score'] = 1- proximity_score[0][0]
  return data

In [13]:
text = batch1[0]

In [30]:
proximity_score_shap(text)

  0%|          | 0/306 [00:00<?, ?it/s]

{'predicted_label': 'Introduction',
 'explanation_label': 'Introduction',
 'proximity score': 0.3146369457244873}

In [31]:
batch1_data = []
batch2_data = []

In [34]:
test_data = df['hindi_text']

In [35]:
for sent in test_data:
  batch1_data.append(proximity_score_shap(sent))

  0%|          | 0/498 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

  0%|          | 0/420 [00:00<?, ?it/s]

  0%|          | 0/420 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

...

  0%|          | 0/498 [00:00<?, ?it/s]

  0%|          | 0/420 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

In [36]:
len(batch1_data)

73

In [37]:
df = pd.DataFrame(batch1_data)

In [38]:
df.to_csv('proximity_score_shap.csv')

In [39]:
df

,predicted_label,explanation_label,proximity score
0,Introduction,Introduction,0.314637
1,Depression,Depression,0.378729
2,Introduction,Introduction,0.258716
3,Depression,Depression,0.407552
4,Grey Area,Grey Area,0.381488
...,...,...,...
68,Introduction,Introduction,0.228905
69,Depression,Grey Area,0.426709
70,Grey Area,Grey Area,0.310608
71,Depression,Depression,0.363244
